<a href="https://colab.research.google.com/github/briocheausucre/PIE-SXS02/blob/main/notebook_de_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Nan nan, j'te l'dis")

Nan nan, j'te l'dis
